In [11]:
# from flightsql import FlightSQLClient
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from influxdb_client import InfluxDBClient, Point, Dialect

import re

from influxdb_client.client.write_api import SYNCHRONOUS

import time
import datetime

import requests

import warnings
from influxdb_client.client.warnings import MissingPivotFunction


In [12]:
#Read config file and load data into the variables

rx_dict = {
    'token': re.compile(r'var token = "(?P<token>.*)"\n'),
    'url': re.compile(r'var url = "(?P<url>.*)"\n'),
    'org': re.compile(r'var org = "(?P<org>.*)"\n'),
    'bucket': re.compile(r'var bucket = "(?P<bucket>.*)"\n'),
}

def _parse_line(line):
    """
    Do a regex search against all defined regexes and
    return the key and match result of the first matching regex

    """
    for key, rx in rx_dict.items():
        match = rx.search(line)
        if match:
            return key, match
    # if there are no matches
    return None, None



filepath = '/root/flexi-pipe/config.go'
# open the file and read through it line by line
with open(filepath, 'r') as file_object:
    line = file_object.readline()
    while line:
        # at each line check for a match with a regex
        key, match = _parse_line(line)

        if key == 'token':
            token = match.group('token')
        elif key == 'url':
            url = match.group('url')
        elif key == 'org':
            org = match.group('org')
        elif key == 'bucket':
            bucket = match.group('bucket')
        
        line = file_object.readline()


url="http://192.168.20.58:8086"

In [18]:
client = InfluxDBClient(url=url, token=token, org=org)

# write_api = client.write_api(write_options=SYNCHRONOUS)
query_api = client.query_api()

In [14]:
# Retrieve experiments data from csv
data = pd.read_csv('./experimentsTest.csv', header=None)
df = pd.DataFrame(data)

#Rename columns
experiments = df.rename(columns={0: "start", 1: "end", 2: "topology", 3: "runtime", 4: "d", 5: "dlo", 6: "dhi", 7: "dscore", 8: "dlazy", 9: "dout", 10: "gossipFactor", 11: "initialDelay", 12: "interval"}, errors='raise')

#Correct timestamp
experiments["start"] = experiments["start"].str.slice(0, 27)
experiments["end"] = experiments["end"].str.slice(0, 27)

#String to timestamp
experiments['startUnix'] = pd.to_datetime(experiments["start"],format="%Y-%m-%d %H:%M:%S.%f").astype('int64') / 10**9
experiments['endUnix'] = pd.to_datetime(experiments["end"],format="%Y-%m-%d %H:%M:%S.%f").astype('int64') / 10**9

experiments['startUnix'] = pd.to_timedelta(experiments['startUnix'], unit='s').dt.total_seconds().astype(int).astype(str)
experiments['endUnix'] = pd.to_timedelta(experiments['endUnix'], unit='s').dt.total_seconds().astype(int).astype(str)

#Drop fields we don't mneed for the moment
exp = experiments.drop(columns=["runtime", "initialDelay"])

exp.head(10)

,start,end,topology,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,startUnix,endUnix
0,2023-07-17 15:40:26.6274422,2023-07-17 15:55:26.6315408,unl,8,6,12,4,8,2,0.25,1.0,1689608426,1689609326
1,2023-07-17 15:57:17.9567294,2023-07-17 16:12:18.0133411,unl,8,6,12,4,8,2,0.25,1.0,1689609437,1689610338
2,2023-07-17 16:14:09.5220844,2023-07-17 16:29:09.5251913,unl,8,6,12,4,8,2,0.25,1.0,1689610449,1689611349
3,2023-07-17 16:31:01.1335124,2023-07-17 16:46:01.1369169,unl,8,6,12,4,8,2,0.25,1.0,1689611461,1689612361
4,2023-07-17 16:47:53.0116256,2023-07-17 17:02:53.0693216,unl,8,6,12,4,8,2,0.25,1.0,1689612473,1689613373
5,2023-07-17 17:04:45.1204716,2023-07-17 17:19:45.1238532,unl,8,6,12,4,8,2,0.25,1.0,1689613485,1689614385
6,2023-07-17 17:21:37.4265269,2023-07-17 17:36:37.4854434,unl,8,6,12,4,8,2,0.25,1.0,1689614497,1689615397
7,2023-07-17 17:38:29.7332263,2023-07-17 17:53:29.7347931,unl,8,6,12,4,8,2,0.25,1.0,1689615509,1689616409
8,2023-07-17 17:55:22.0235966,2023-07-17 18:10:22.0809455,unl,8,6,12,4,8,2,0.25,1.0,1689616522,1689617422
9,2023-07-17 18:12:14.7230670,2023-07-17 18:27:14.7259267,unl,12,6,12,4,8,2,0.25,1.0,1689617534,1689618434


In [15]:
# Select only the last 6 experiments

# REMOVE THIS AFTER FINISH DEV
exp = exp.tail(6)

exp.head(6)

,start,end,topology,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,startUnix,endUnix
67,2023-07-20 12:59:23.6641454,2023-07-20 13:14:23.6682449,unl,8,6,12,4,8,2,0.25,0.5,1689857963,1689858863
68,2023-07-20 13:16:15.4454777,2023-07-20 13:31:15.5053115,unl,8,6,12,4,8,2,0.25,0.5,1689858975,1689859875
69,2023-07-20 13:33:07.7805999,2023-07-20 13:48:07.7828809,unl,8,6,12,4,8,2,0.25,0.5,1689859987,1689860887
70,2023-07-20 13:49:59.8172298,2023-07-20 14:04:59.8203737,unl,8,6,12,4,8,2,0.25,1.0,1689860999,1689861899
71,2023-07-20 14:06:52.4608735,2023-07-20 14:21:52.4646025,unl,8,6,12,4,8,2,0.25,1.0,1689862012,1689862912
72,2023-07-20 14:23:45.3675268,2023-07-20 14:38:45.3752391,unl,8,6,12,4,8,2,0.25,1.0,1689863025,1689863925


In [19]:
# exp1 = exp.head(1).reset_index(drop=True) 
# start = exp1["startUnix"]
# end = exp1["endUnix"]
# warnings.warn(message, MissingPivotFunction)

data_frame = query_api.query_data_frame('from(bucket: "gs") '
                                        '|> range(start: -7d) '
                                        '|> filter(fn: (r) => r._measurement == "message" '
                                        '|> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")')
# print(data_frame.to_string())

client.close()

ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Content-Type': 'application/json; charset=utf-8', 'Vary': 'Accept-Encoding', 'X-Influxdb-Build': 'OSS', 'X-Influxdb-Version': 'v2.7.0', 'X-Platform-Error-Code': 'invalid', 'Date': 'Mon, 24 Jul 2023 12:22:51 GMT', 'Transfer-Encoding': 'chunked'})
HTTP response body: b'{"code":"invalid","message":"compilation failed: error @1:44-1:162: expected RPAREN, got EOF"}'


In [ ]:
data_frame.head(15)